### Lab 11 MNIST and Deep learning CNN

In [4]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets('MNIST_data/', one_hot = True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [5]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [9]:
tf.reset_default_graph()
class Model:
    def __init__(self, sess, name):
        self.sess = sess
        self.name = name
        self._build_net()
    
    def _build_net(self):
        with tf.variable_scope(self.name):
            self.keep_prob = tf.placeholder(tf.float32)
        
            self.X = tf.placeholder(tf.float32, [None, 784])
            X_img = tf.reshape(self.X, [-1, 28, 28, 1])
            self.Y = tf.placeholder(tf.float32, [None, 10])
            
            W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev = 0.001))
            L1 = tf.nn.conv2d(X_img, W1, strides = [1,1,1,1], padding = 'SAME')
            L1 = tf.nn.relu(L1)
            L1 = tf.nn.max_pool(L1, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'SAME')
            L1 = tf.nn.dropout(L1, keep_prob = self.keep_prob)
            
            W2 = tf.Variable(tf.random_normal([3,3,32,64], stddev = 0.01))
            L2 = tf.nn.conv2d(L1, W2, strides = [1,1,1,1], padding = 'SAME')
            L2 = tf.nn.relu(L2)
            L2 = tf.nn.max_pool(L2, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'SAME')
            L2 = tf.nn.dropout(L2, keep_prob = self.keep_prob)
            
            W3 = tf.Variable(tf.random_normal([3,3,64,128], stddev = 0.01))
            L3 = tf.nn.conv2d(L2, W3, strides = [1,1,1,1], padding = 'SAME')
            L3 = tf.nn.relu(L3)
            L3 = tf.nn.max_pool(L3,ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'SAME')
            L3 = tf.nn.dropout(L3, keep_prob = self.keep_prob)
            L3 = tf.reshape(L3, [-1, 128*4*4])
            
            W4 = tf.get_variable('W4', shape = [128*4*4, 625],
                                initializer = tf.contrib.layers.xavier_initializer())
            b4 = tf.Variable(tf.random_normal([625]))
            L4 = tf.nn.relu(tf.matmul(L3, W4) + b4)
            L4 = tf.nn.dropout(L4, keep_prob = self.keep_prob)
            
            W5 = tf.get_variable('W5', shape = [625, 10],
                                initializer = tf.contrib.layers.xavier_initializer())
            b5 = tf.Variable(tf.random_normal([10]))
            self.hypothesis = tf.matmul(L4, W5) + b5
        
        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
            logits = self.hypothesis, labels = self.Y))
        self.optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(self.cost)
        
        correct_prediction = tf.equal(tf.argmax(self.hypothesis, 1), tf.argmax(self.Y, 1))
        self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    def predict(self, x_test, keep_prop = 1.0):
        return self.sess.run(self.hypothesis, feed_dict = {self.X: x_test, self.keep_prob : keep_prop})
    
    def get_accuracy(self, x_test, y_data, keep_prop = 1.0):
        return self.sess.run(self.accuracy, feed_dict = {self.X : x_test, self.Y: y_data, self.keep_prob: keep_prop})
    
    def train(self, x_data, y_data, keep_prop = 0.7):
        return self.sess.run([self.cost, self.optimizer], feed_dict = {self.X: x_data, self.Y: y_data, self.keep_prob: keep_prop})
    
with tf.Session() as sess:
    m1 = Model(sess, 'm1')
    
    sess.run(tf.global_variables_initializer())
    print('Learning Started!')
    
    for epoch in range(training_epochs):
        avg_cost = 0
        total_batch = int(mnist.train.num_examples / batch_size)
        
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            c, _ = m1.train(batch_xs, batch_ys)
            avg_cost += c / total_batch
        
        print('Epoch: ', '%04d' %(epoch + 1), 'cost = ', '{:.9f}'.format(avg_cost))
    print('Learning Finished!')
    
    print('Accuracy:', m1.get_accuracy(mnist.test.images, mnist.test.labels))

Learning Started!
Epoch:  0001 cost =  0.519240854
Epoch:  0002 cost =  0.117952770
Epoch:  0003 cost =  0.085528902
Epoch:  0004 cost =  0.071757726
Epoch:  0005 cost =  0.059792962
Epoch:  0006 cost =  0.054299679
Epoch:  0007 cost =  0.050302580
Epoch:  0008 cost =  0.047111661
Epoch:  0009 cost =  0.040391815
Epoch:  0010 cost =  0.040167259
Epoch:  0011 cost =  0.037502167
Epoch:  0012 cost =  0.034442229
Epoch:  0013 cost =  0.033991224
Epoch:  0014 cost =  0.033043144
Epoch:  0015 cost =  0.030145450
Learning Finished!
Accuracy: 0.9925


### MNIST CNN LAYERS

In [10]:
tf.reset_default_graph()

mnist = input_data.read_data_sets('MNIST_data/', one_hot = True)

learning_rate = 0.001
training_epochs = 15
batch_size = 100

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [11]:
class Model:
    
    def __init__(self, sess, name):
        self.sess = sess
        self.name = name
        self._build_net()
        
    def _build_net(self):
        with tf.variable_scope(self.name):
            
            self.training = tf.placeholder(tf.bool)
            self.X = tf.placeholder(tf.float32, [None, 784])
            
            X_img = tf.reshape(self.X, [-1, 28, 28, 1])
            self.Y = tf.placeholder(tf.float32, [None, 10])
            
            conv1 = tf.layers.conv2d(inputs = X_img, filters = 32, kernel_size = [3,3],
                                    padding = 'SAME', activation = tf.nn.relu)
            pool1 = tf.layers.max_pooling2d(inputs = conv1, pool_size = [2,2],
                                           padding = 'SAME', strides = 2)
            dropout1 = tf.layers.dropout(inputs = pool1, rate = 0.3, training = self.training)
            
            conv2  = tf.layers.conv2d(inputs = dropout1, filters = 64, kernel_size = [3,3],
                                     padding = 'SAME', activation = tf.nn.relu)
            pool2 = tf.layers.max_pooling2d(inputs = conv2, pool_size = [2,2],
                                           padding = 'SAME', strides = 2)
            dropout2 = tf.layers.dropout(inputs = pool2, rate = 0.3, training = self.training)
            
            conv3 = tf.layers.conv2d(inputs = dropout2, filters = 128, kernel_size = [3,3],
                                    padding = 'SAME', activation = tf.nn.relu)
            pool3 = tf.layers.max_pooling2d(inputs = conv3, pool_size = [2,2],
                                           padding = 'SAME', strides = 2)
            dropout3 = tf.layers.dropout(inputs = pool3, rate = 0.3, training = self.training)
            
            flat = tf.reshape(dropout3, [-1, 128 * 4 * 4])
            dense4 = tf.layers.dense(inputs = flat, units = 625, activation = tf.nn.relu)
            dropout4 = tf.layers.dropout(inputs = dense4, rate = 0.5, training = self.training)
            
            self.hypothesis = tf.layers.dense(inputs = dropout4, units = 10)
        
        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
        logits = self.hypothesis, labels = self.Y))
        self.optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(self.cost)
        
        correct_prediction = tf.equal(
        tf.argmax(self.hypothesis, 1), tf.argmax(self.Y, 1))
        
        self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    def predict(self, x_test, training = False):
        return self.sess.run(self.hypothesis,
                            feed_dict = {self.X: x_test, self.training: training})
    def get_accuracy(self, x_test, y_test, training = False):
        return self.sess.run(self.accuracy,
                            feed_dict = {self.X: x_test, self.Y: y_test, self.training: training})
    def train(self, x_data, y_data, training = True):
        return self.sess.run([self.cost, self.optimizer],
                            feed_dict = {self.X: x_data, self.Y: y_data, self.training: training})
    
with tf.Session() as sess:
    m1 = Model(sess, 'm1')
    sess.run(tf.global_variables_initializer())
    
    print('Learning Started!')
    
    for epoch in range(training_epochs):
        avg_cost = 0
        total_batch = int(mnist.train.num_examples / batch_size)
        
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            c, _ = m1.train(batch_xs, batch_ys)
            avg_cost += c / total_batch
            
        print('Epoch: ', '%04d' %(epoch + 1), 'cost = ', '{:.9f}'.format(avg_cost))
    
    print('Learning Finished!')
    
    print('Accuracy: ', m1.get_accuracy(mnist.test.images, mnist.test.labels))
        

Learning Started!
Epoch:  0001 cost =  0.288158098
Epoch:  0002 cost =  0.087676250
Epoch:  0003 cost =  0.068825883
Epoch:  0004 cost =  0.054651431
Epoch:  0005 cost =  0.049024772
Epoch:  0006 cost =  0.045238486
Epoch:  0007 cost =  0.040824969
Epoch:  0008 cost =  0.039599644
Epoch:  0009 cost =  0.036183046
Epoch:  0010 cost =  0.033887589
Epoch:  0011 cost =  0.031045233
Epoch:  0012 cost =  0.030721111
Epoch:  0013 cost =  0.028803698
Epoch:  0014 cost =  0.027675847
Epoch:  0015 cost =  0.027847916
Learning Finished!
Accuracy:  0.994


### MNIST CNN ensemble layers

In [12]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets('MNIST_data/', one_hot = True)
tf.reset_default_graph()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [13]:
learnin_rate = 0.001
training_epochs = 20
batch_size = 100

In [14]:
class Model:
    
    def __init__(self, sess, name):
        self.sess = sess
        self.name = name
        self._build_net()
    
    def _build_net(self):
        with tf.variable_scope(self.name):
            self.training = tf.placeholder(tf.bool)
        
            self.X = tf.placeholder(tf.float32, [None, 784])

            X_img = tf.reshape(self.X, [-1, 28, 28, 1])
            self.Y = tf.placeholder(tf.float32, [None, 10])

            conv1 = tf.layers.conv2d(inputs = X_img, filters = 32, kernel_size = [3,3],
                                    padding = 'SAME', activation = tf.nn.relu)
            pool1 = tf.layers.max_pooling2d(inputs = conv1, pool_size = [2,2],
                                           padding = 'SAME', strides = 2)
            dropout1 = tf.layers.dropout(inputs = pool1, rate = 0.3, training = self.training)

            conv2 = tf.layers.conv2d(inputs = dropout1, filters = 64, kernel_size = [3,3],
                                    padding = 'SAME', activation = tf.nn.relu)
            pool2 = tf.layers.max_pooling2d(inputs = conv2, pool_size = [2,2],
                                         padding = 'SAME', strides = 2)
            dropout2 = tf.layers.dropout(inputs = pool2, rate = 0.3, training = self.training)

            conv3 = tf.layers.conv2d(inputs = dropout2, filters = 128, kernel_size = [3,3],
                                    padding = 'SAME', activation = tf.nn.relu)
            pool3 = tf.layers.max_pooling2d(inputs = conv3, pool_size = [2,2],
                                           padding = 'SAME', strides = 2)
            dropout3 = tf.layers.dropout(inputs = pool3, rate = 0.3, training = self.training)

            flat = tf.reshape(dropout3, [-1, 128*4*4])
            dense4 = tf.layers.dense(inputs = flat, units = 625, activation = tf.nn.relu)
            dropout4 = tf.layers.dropout(inputs = dense4, rate = 0.5, training = self.training)

            self.hypothesis = tf.layers.dense(inputs = dropout4, units = 10)

        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
        logits = self.hypothesis, labels = self.Y))
        self.optimizer = tf.train.AdamOptimizer(
        learning_rate = learning_rate).minimize(self.cost)

        correct_prediction = tf.equal(tf.argmax(self.hypothesis, 1), tf.argmax(self.Y, 1))
        self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    def predict(self, x_test, training = False):
        return self.sess.run(self.hypothesis, feed_dict = {self.X: x_test, self.training: training})

    def get_accuracy(self, x_test, y_test, training = False):
        return self.sess.run(self.accuracy, feed_dict = {self.X: x_test, 
                                                        self.Y: y_test, self.training : training})

    def train(self, x_data, y_data, training = True):
        return self.sess.run([self.cost, self.optimizer], feed_dict = {
            self.X: x_data, self.Y: y_data, self.training: training})

with tf.Session() as sess:
    models = []
    num_models = 2
    for m in range(num_models):
        models.append(Model(sess, 'model' + str(m)))
    
    sess.run(tf.global_variables_initializer())
    print('Learning Started!')
    
    for epoch in range(training_epochs):
        avg_cost_list = np.zeros(len(models))
        total_batch = int(mnist.train.num_examples / batch_size)
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            
            for m_idx, m in enumerate(models):
                c, _ = m.train(batch_xs, batch_ys)
                avg_cost_list[m_idx] += c / total_batch
        print('Epoch: ', '%04d' %(epoch + 1), 'cost = ', avg_cost_list)
    print('Learning Finished!')
    
    test_size = len(mnist.test.labels)
    predictions = np.zeros([test_size, 10])
    
    for m_idx, m in enumerate(models):
        print(m_idx, 'Accuracy: ', m.get_accuracy(
        mnist.test.images, mnist.test.labels))
        p = m.predict(mnist.test.images)
        predictions += p
        
    ensemble_correct_prediction = tf.equal(
    tf.argmax(predictions, 1), tf.argmax(mnist.test.labels, 1))
    ensemble_accuracy = tf.reduce_mean(tf.cast(ensemble_correct_prediction, tf.float32))
    print('Ensemble accuracy: ', sess.run(ensemble_accuracy))

Learning Started!
Epoch:  0001 cost =  [0.28504319 0.29125041]
Epoch:  0002 cost =  [0.08876877 0.08888642]
Epoch:  0003 cost =  [0.06718016 0.06943495]
Epoch:  0004 cost =  [0.05381661 0.0562987 ]
Epoch:  0005 cost =  [0.0489313  0.05060171]
Epoch:  0006 cost =  [0.04262371 0.04609001]
Epoch:  0007 cost =  [0.03947154 0.04178705]
Epoch:  0008 cost =  [0.03610835 0.03644438]
Epoch:  0009 cost =  [0.03366635 0.03598319]
Epoch:  0010 cost =  [0.03264158 0.03401547]
Epoch:  0011 cost =  [0.03066714 0.02949062]
Epoch:  0012 cost =  [0.02950711 0.03149674]
Epoch:  0013 cost =  [0.03078607 0.02909082]
Epoch:  0014 cost =  [0.02666951 0.02730476]
Epoch:  0015 cost =  [0.02559192 0.02660294]
Epoch:  0016 cost =  [0.02486477 0.02828554]
Epoch:  0017 cost =  [0.02537297 0.0253453 ]
Epoch:  0018 cost =  [0.02312161 0.02428287]
Epoch:  0019 cost =  [0.02260424 0.0222857 ]
Epoch:  0020 cost =  [0.02343761 0.02260777]
Learning Finished!
0 Accuracy:  0.9936
1 Accuracy:  0.9943
Ensemble accuracy:  0.9